In [1]:
import pandas as pd
import numpy as np
import random
import time
# https://pypi.org/project/progressbar2/
import progressbar
import seaborn as sns
#mport matplotlib
import matplotlib.pyplot as plt

In [2]:
class RankChoice:

    def __init__(self, config):
        np.random.seed(config["r_seed"]) 
        random.seed(config["r_seed"])
        self.candidates = config["candidates"]
        self.pop_size = config["pop_size"]
        self.per_of_turnout = config["turnout_per"]
        self.print_on = config["print_on"]
        self.require_pref = config["require_all_pref"]
        self.votes_pop = pd.DataFrame(index=range(self.pop_size), columns=["Favorite", "Second Choice", "Eh", "Only Cause I Have To", "No Chance"]).fillna(-1)
        self.votes_cast = pd.DataFrame.empty
        self.vote_count = {}
        self.initial_results = {}
        self.eliminated_candidates = []
        self.runoffs = 0
        self.total_votes = 0
        self.rank_margin = 0.0
        self.common_margin = 0.0
        self.rank_fav_win = True
        self.common_fav_win = True

    def run(self):
        """Run the program."""
        self.getVotePopulation()
        self.dropTurnout()
        self.getVoteCount()
        self.setTotalVotes()
        self.commonElection()
        self.rankChoice()

    def getProbs(self):
        """Return weighted probabilities of each candidate."""
        return [self.candidates[i][1] for i in range(len(self.candidates.keys()))]

    # FOR DEMONSTRATION PURPOSES
    # newl = np.random.choice([0, 1, 2, 3, 4], size=5, replace=False, p=[0.96, 0.01, 0.01, 0.01, 0.01])
    # print(newl) --> [[0 4 2 1 3], [0 2 4 1 3], [0 2 4 3 1], [0 3 2 4 1], [0 3 1 4 2]]
    def getVotePopulation(self):
        """create vote population."""
        vote_list = range(len(self.candidates.keys()))
        ps = self.getProbs()
        for i in range(self.pop_size):
            if self.require_pref:
                newl = np.random.choice(vote_list, size=len(vote_list), replace=False, p=ps)
            else:
                # get random number of vote preference
                num_votes = random.randrange(1, len(self.candidates.keys()))
                newl = np.random.choice(vote_list, size=num_votes, replace=False, p=ps)
            v_size = len(newl)
            self.votes_pop.at[i, "Favorite"] = newl[0]
            self.votes_pop.at[i, "Second Choice"] = newl[1] if v_size > 1 else -1
            self.votes_pop.at[i, "Eh"] = newl[2] if v_size > 2 else -1
            self.votes_pop.at[i, "Only Cause I Have To"] = newl[3] if v_size > 3 else -1
            self.votes_pop.at[i, "No Chance"] = newl[4] if v_size > 4 else -1

    def getVoteCount(self):
        """Count votes."""
        for i in range(len(self.candidates)):
            indices = self.votes_cast.loc[ (self.votes_cast["Favorite"].iloc[:] == i) | (self.votes_cast["Favorite"].iloc[:] == float(i))].index.values
            self.vote_count[i] = len(indices)
        self.vote_count = dict(sorted(self.vote_count.items(), key=lambda x: x[1], reverse=True))
        if self.print_on:
            self.printResults()

    def printResults(self):
        """Print out the results of the vote count."""
        print({self.candidates[key][0]:value for key,value in self.vote_count.items()})

    def dropTurnout(self):
        """Remove percentage of votes as non-turnout voters."""
        no_vote = np.random.choice(range(self.votes_pop.shape[0]), size=int(self.votes_pop.shape[0]*self.per_of_turnout), replace=False)
        self.votes_cast = self.votes_pop.drop(no_vote).astype(int)

    def rankChoice(self):
        """Return results of the election based on rank choice rules."""
        half_plus_one = int(self.total_votes/2) + 1
        top_total_votes = list(self.vote_count.values())[0]
        self.rank_margin = abs(list(self.vote_count.values())[0] - list(self.vote_count.values())[1])
        if top_total_votes >= half_plus_one:
            # if person with top votes gets a majority, they win
            if self.candidates[list(self.vote_count.keys())[0]][0] != self.candidates[0][0]:
                # if candidate that wins is not fav candidate, switch this false
                self.rank_fav_win = False
            if self.print_on:
                print(self.candidates[list(self.vote_count.keys())[0]][0] + " Wins Rank Choice Election! : " + str(top_total_votes/self.total_votes))
        else:
            self.runoff()

    def runoff(self):
        """Candidate with fewest first-preference votes is eliminated."""
        self.runoffs = self.runoffs + 1
        few_cand = self.getFewestCandidate()
        self.updateVotesCast(few_cand)
        self.getVoteCount()
        self.setTotalVotes()
        self.rankChoice()

    def setTotalVotes(self):
        """Set the total number of votes."""
        self.total_votes = sum(self.vote_count.values())

    def getFewestCandidate(self):
        """Get candidate with fewest first-preference votes."""
        fewest_cand = int(self.votes_cast["Favorite"].value_counts().index.values[-1])
        self.eliminated_candidates.append(fewest_cand)
        return fewest_cand

    def updateVotesCast(self, elim_candidate):
        """Update votes_cast df based on eliminated candidate."""
        for i in range(self.votes_cast.shape[0]):
            self.checkForElimVotes(i, elim_candidate)

    def checkForElimVotes(self, i, elim_candidate):
        """Checks votes for individual row to see if next vote is for an eliminated candidate."""
        v1 = self.votes_cast["Favorite"].iloc[i]
        if v1 == elim_candidate or v1 in self.eliminated_candidates:
            self.shiftVotes(i)
            self.checkForElimVotes(i, elim_candidate)   # call function recursively to make sure newly shifted v1 vote is not for an eliminated candidate

    def shiftVotes(self, i):
        """Shift given row of votes by one to the left."""
        self.votes_cast.iloc[i, :] = pd.Series(self.votes_cast.iloc[i,:]).shift(-1)

    def commonElection(self):
        """Run the common election simulation."""
        top_total_votes = list(self.vote_count.values())[0]
        self.common_margin = abs(list(self.vote_count.values())[0] - list(self.vote_count.values())[1])
        if self.print_on:
            print(self.candidates[list(self.vote_count.keys())[0]][0] + " Wins Common Election! : " + str(top_total_votes/self.total_votes))


In [3]:
config = {
    "pop_size": 1000,
    "print_on": True,
    "turnout_per": 0.5,
    "require_all_pref": False,
    "r_seed": 2,
    "candidates": {
        0: ["Elizabeth II", 0.36], 
        1: ["Genghis Khan", 0.35], 
        2: ["Alexander the Great", 0.13], 
        3: ["Mahatma Gandhi", 0.11], 
        4: ["Augustus Caesar", 0.05]
    }
}

rc = RankChoice(config)
rc.run()

{'Elizabeth II': 186, 'Genghis Khan': 173, 'Alexander the Great': 63, 'Mahatma Gandhi': 47, 'Augustus Caesar': 31}
Elizabeth II Wins Common Election! : 0.372
{'Elizabeth II': 192, 'Genghis Khan': 182, 'Alexander the Great': 67, 'Mahatma Gandhi': 52, 'Augustus Caesar': 0}
{'Elizabeth II': 192, 'Genghis Khan': 182, 'Alexander the Great': 67, 'Mahatma Gandhi': 52, 'Augustus Caesar': 0}
{'Genghis Khan': 203, 'Elizabeth II': 201, 'Alexander the Great': 75, 'Mahatma Gandhi': 0, 'Augustus Caesar': 0}
{'Genghis Khan': 233, 'Elizabeth II': 232, 'Alexander the Great': 0, 'Mahatma Gandhi': 0, 'Augustus Caesar': 0}
Genghis Khan Wins Rank Choice Election! : 0.5010752688172043


# Experiments on Effects of Turnout

# Rank Choice Voting vs. Common Election

In [9]:
n = 1000

results = pd.DataFrame(index=range(n), columns=["rank_margins", "rank_margins_pers", "rank_runoffs", "rank_fav_wins", "common_margins", "common_margins_pers", "common_fav_wins"])
for i in progressbar.progressbar(range(n)):
    next_config = {
        "pop_size": 10000,
        "print_on": False,
        "turnout_per": 0.5,
        "require_all_pref": False,
        "r_seed": i,
        "candidates": {
            0: ["Elizabeth II", 0.46], 
            1: ["Genghis Khan", 0.25], 
            2: ["Alexander the Great", 0.13], 
            3: ["Mahatma Gandhi", 0.11], 
            4: ["Augustus Caesar", 0.05]
        }
    }

    rc = RankChoice(next_config)
    rc.run()

    results.at[i, "rank_runoffs"] = rc.runoffs

    results.at[i, "rank_margins"] = rc.rank_margin
    results.at[i, "rank_margins_pers"] = round(rc.rank_margin/rc.total_votes, 3)
    results.at[i, "common_margins"] = rc.common_margin
    results.at[i, "common_margins_pers"] = round(rc.common_margin/rc.total_votes, 3)

    results.at[i, "rank_fav_wins"] = rc.rank_fav_win
    results.at[i, "common_fav_wins"] = rc.common_fav_win

rfws = results["rank_fav_wins"].sum()
print(f"Rank Choice Election Fav. Wins Sum: {rfws}/{n}")
cfws = results["common_fav_wins"].sum()
print(f"Common Election Fav. Wins Sum: {cfws}/{n}")

rm = round(results["rank_margins"].mean(), 3)
print(f"Rank Choice Election Win Margin: {rm}")
rpm = round(results["rank_margins_pers"].mean(), 3)
print(f"Rank Choice  Election Win Margin Percentage: {rpm}%")

cm = round(results["common_margins"].mean(), 3)
print(f"Common Election Win Margin: {cm}")
cpm = round(results["common_margins_pers"].mean(), 3)
print(f"Common Election Win Margin Percentage: {cpm}%")


sns.histplot(data=results, x="rank_margins")
plt.show()
sns.histplot(data=results, x="rank_margins_pers")
plt.show()
sns.histplot(data=results, x="common_margins")
plt.show()
sns.histplot(data=results, x="common_margins_pers")
plt.show()
sns.histplot(data=results, x="rank_runoffs")
plt.show()


  0% (1 of 1000) |                       | Elapsed Time: 0:00:01 ETA:   0:21:04

KeyboardInterrupt: 

# Required Preference vs. Not Required Preference

In [10]:
n = 1000

results1 = pd.DataFrame(index=range(n), columns=["rank_margins", "rank_margins_pers", "rank_runoffs", "rank_fav_wins"])
results2 = pd.DataFrame(index=range(n), columns=["rank_margins", "rank_margins_pers", "rank_runoffs", "rank_fav_wins"])
for i in progressbar.progressbar(range(n)):
    next_config = {
        "pop_size": 10000,
        "print_on": False,
        "turnout_per": 0.5,
        "require_all_pref": False,
        "r_seed": i,
        "candidates": {
            0: ["Elizabeth II", 0.46], 
            1: ["Genghis Khan", 0.25], 
            2: ["Alexander the Great", 0.13], 
            3: ["Mahatma Gandhi", 0.11], 
            4: ["Augustus Caesar", 0.05]
        }
    }

    rc = RankChoice(next_config)
    rc.run()

    results1.at[i, "rank_runoffs"] = rc.runoffs

    results1.at[i, "rank_margins"] = rc.rank_margin
    results1.at[i, "rank_margins_pers"] = round(rc.rank_margin/rc.total_votes, 3)

    results1.at[i, "rank_fav_wins"] = rc.rank_fav_win

    next_config = {
        "pop_size": 10000,
        "print_on": False,
        "turnout_per": 0.5,
        "require_all_pref": True,
        "r_seed": i,
        "candidates": {
            0: ["Elizabeth II", 0.46], 
            1: ["Genghis Khan", 0.25], 
            2: ["Alexander the Great", 0.13], 
            3: ["Mahatma Gandhi", 0.11], 
            4: ["Augustus Caesar", 0.05]
        }
    }

    rc = RankChoice(next_config)
    rc.run()

    results2.at[i, "rank_runoffs"] = rc.runoffs

    results2.at[i, "rank_margins"] = rc.rank_margin
    results2.at[i, "rank_margins_pers"] = round(rc.rank_margin/rc.total_votes, 3)

    results2.at[i, "rank_fav_wins"] = rc.rank_fav_win

rfws = results1["rank_fav_wins"].sum()
print(f"Not Required Preference Rank Choice Election Fav. Wins Sum: {rfws}/{n}")

rm = round(results1["rank_margins"].mean(), 3)
print(f"Not Required Preference Rank Choice Election Win Margin: {rm}")
rpm = round(results1["rank_margins_pers"].mean(), 3)
print(f"Not Required Preference Rank Choice  Election Win Margin Percentage: {rpm}%")


rfws = results2["rank_fav_wins"].sum()
print(f"Required Preference Rank Choice Election Fav. Wins Sum: {rfws}/{n}")

rm = round(results2["rank_margins"].mean(), 3)
print(f"Required Preference Rank Choice Election Win Margin: {rm}")
rpm = round(results2["rank_margins_pers"].mean(), 3)
print(f"Required Preference Rank Choice  Election Win Margin Percentage: {rpm}%")


sns.histplot(data=results1, x="rank_margins")
plt.show()
sns.histplot(data=results1, x="rank_margins_pers")
plt.show()
sns.histplot(data=results1, x="rank_runoffs")
plt.show()

sns.histplot(data=results2, x="rank_margins")
plt.show()
sns.histplot(data=results2, x="rank_margins_pers")
plt.show()
sns.histplot(data=results2, x="rank_runoffs")
plt.show()

  0% (1 of 1000) |                       | Elapsed Time: 0:00:02 ETA:   0:41:42

KeyboardInterrupt: 

# Variable Number of Candidates

In [11]:
n = 1000

results1 = pd.DataFrame(index=range(n), columns=["rank_margins", "rank_margins_pers", "rank_runoffs", "rank_fav_wins"])
results2 = pd.DataFrame(index=range(n), columns=["rank_margins", "rank_margins_pers", "rank_runoffs", "rank_fav_wins"])
results3 = pd.DataFrame(index=range(n), columns=["rank_margins", "rank_margins_pers", "rank_runoffs", "rank_fav_wins"])
results4 = pd.DataFrame(index=range(n), columns=["rank_margins", "rank_margins_pers", "rank_runoffs", "rank_fav_wins"])
for i in progressbar.progressbar(range(n)):
    next_config = {
        "pop_size": 10000,
        "print_on": False,
        "turnout_per": 0.5,
        "require_all_pref": False,
        "r_seed": i,
        "candidates": {
            0: ["Elizabeth II", 0.53], 
            1: ["Genghis Khan", 0.47]
        }
    }

    rc = RankChoice(next_config)
    rc.run()

    results1.at[i, "rank_runoffs"] = rc.runoffs

    results1.at[i, "rank_margins"] = rc.rank_margin
    results1.at[i, "rank_margins_pers"] = round(rc.rank_margin/rc.total_votes, 3)

    results1.at[i, "rank_fav_wins"] = rc.rank_fav_win

    next_config = {
        "pop_size": 10000,
        "print_on": False,
        "turnout_per": 0.5,
        "require_all_pref": False,
        "r_seed": i,
        "candidates": {
            0: ["Elizabeth II", 0.49], 
            1: ["Genghis Khan", 0.33], 
            2: ["Alexander the Great", 0.18]
        }
    }

    rc = RankChoice(next_config)
    rc.run()

    results2.at[i, "rank_runoffs"] = rc.runoffs

    results2.at[i, "rank_margins"] = rc.rank_margin
    results2.at[i, "rank_margins_pers"] = round(rc.rank_margin/rc.total_votes, 3)

    results2.at[i, "rank_fav_wins"] = rc.rank_fav_win

    next_config = {
        "pop_size": 10000,
        "print_on": False,
        "turnout_per": 0.5,
        "require_all_pref": False,
        "r_seed": i,
        "candidates": {
            0: ["Elizabeth II", 0.46], 
            1: ["Genghis Khan", 0.30], 
            2: ["Alexander the Great", 0.13], 
            3: ["Mahatma Gandhi", 0.11]
        }
    }

    rc = RankChoice(next_config)
    rc.run()

    results3.at[i, "rank_runoffs"] = rc.runoffs

    results3.at[i, "rank_margins"] = rc.rank_margin
    results3.at[i, "rank_margins_pers"] = round(rc.rank_margin/rc.total_votes, 3)

    results3.at[i, "rank_fav_wins"] = rc.rank_fav_win

    next_config = {
        "pop_size": 10000,
        "print_on": False,
        "turnout_per": 0.5,
        "require_all_pref": False,
        "r_seed": i,
        "candidates": {
            0: ["Elizabeth II", 0.46], 
            1: ["Genghis Khan", 0.25], 
            2: ["Alexander the Great", 0.13], 
            3: ["Mahatma Gandhi", 0.11], 
            4: ["Augustus Caesar", 0.05]
        }
    }

    rc = RankChoice(next_config)
    rc.run()

    results4.at[i, "rank_runoffs"] = rc.runoffs

    results4.at[i, "rank_margins"] = rc.rank_margin
    results4.at[i, "rank_margins_pers"] = round(rc.rank_margin/rc.total_votes, 3)

    results4.at[i, "rank_fav_wins"] = rc.rank_fav_win

rfws = results1["rank_fav_wins"].sum()
print(f"1 Candidate Rank Choice Election Fav. Wins Sum: {rfws}/{n}")

rm = round(results1["rank_margins"].mean(), 3)
print(f"1 Candidate Rank Choice Election Win Margin: {rm}")
rpm = round(results1["rank_margins_pers"].mean(), 3)
print(f"1 Candidate Rank Choice  Election Win Margin Percentage: {rpm}%")


rfws = results2["rank_fav_wins"].sum()
print(f"2 Candidates Rank Choice Election Fav. Wins Sum: {rfws}/{n}")

rm = round(results2["rank_margins"].mean(), 3)
print(f"2 Candidates Rank Choice Election Win Margin: {rm}")
rpm = round(results2["rank_margins_pers"].mean(), 3)
print(f"2 Candidates Rank Choice  Election Win Margin Percentage: {rpm}%")


rfws = results3["rank_fav_wins"].sum()
print(f"3 Candidates Rank Choice Election Fav. Wins Sum: {rfws}/{n}")

rm = round(results3["rank_margins"].mean(), 3)
print(f"3 Candidates Rank Choice Election Win Margin: {rm}")
rpm = round(results3["rank_margins_pers"].mean(), 3)
print(f"3 Candidates Rank Choice  Election Win Margin Percentage: {rpm}%")


rfws = results4["rank_fav_wins"].sum()
print(f"4 Candidates Rank Choice Election Fav. Wins Sum: {rfws}/{n}")

rm = round(results4["rank_margins"].mean(), 3)
print(f"4 Candidates Rank Choice Election Win Margin: {rm}")
rpm = round(results4["rank_margins_pers"].mean(), 3)
print(f"4 Candidates Rank Choice  Election Win Margin Percentage: {rpm}%")


sns.histplot(data=results1, x="rank_margins")
plt.show()
sns.histplot(data=results1, x="rank_margins_pers")
plt.show()
sns.histplot(data=results1, x="rank_runoffs")
plt.show()

sns.histplot(data=results2, x="rank_margins")
plt.show()
sns.histplot(data=results2, x="rank_margins_pers")
plt.show()
sns.histplot(data=results2, x="rank_runoffs")
plt.show()

sns.histplot(data=results3, x="rank_margins")
plt.show()
sns.histplot(data=results3, x="rank_margins_pers")
plt.show()
sns.histplot(data=results3, x="rank_runoffs")
plt.show()

sns.histplot(data=results4, x="rank_margins")
plt.show()
sns.histplot(data=results4, x="rank_margins_pers")
plt.show()
sns.histplot(data=results4, x="rank_runoffs")
plt.show()

  0% (1 of 1000) |                       | Elapsed Time: 0:00:03 ETA:   1:04:52

KeyboardInterrupt: 

# Variable Turnout Percentage

In [12]:
n = 1000

results1 = pd.DataFrame(index=range(n), columns=["rank_margins", "rank_margins_pers", "rank_runoffs", "rank_fav_wins"])
results2 = pd.DataFrame(index=range(n), columns=["rank_margins", "rank_margins_pers", "rank_runoffs", "rank_fav_wins"])
for i in progressbar.progressbar(range(n)):
    next_config = {
        "pop_size": 10000,
        "print_on": False,
        "turnout_per": 0.5,
        "require_all_pref": False,
        "r_seed": i,
        "candidates": {
            0: ["Elizabeth II", 0.46], 
            1: ["Genghis Khan", 0.25], 
            2: ["Alexander the Great", 0.13], 
            3: ["Mahatma Gandhi", 0.11], 
            4: ["Augustus Caesar", 0.05]
        }
    }

    rc = RankChoice(next_config)
    rc.run()

    results1.at[i, "rank_runoffs"] = rc.runoffs

    results1.at[i, "rank_margins"] = rc.rank_margin
    results1.at[i, "rank_margins_pers"] = round(rc.rank_margin/rc.total_votes, 3)

    results1.at[i, "rank_fav_wins"] = rc.rank_fav_win

    next_config = {
        "pop_size": 10000,
        "print_on": False,
        "turnout_per": 0.25,
        "require_all_pref": False,
        "r_seed": i,
        "candidates": {
            0: ["Elizabeth II", 0.46], 
            1: ["Genghis Khan", 0.25], 
            2: ["Alexander the Great", 0.13], 
            3: ["Mahatma Gandhi", 0.11], 
            4: ["Augustus Caesar", 0.05]
        }
    }

    rc = RankChoice(next_config)
    rc.run()

    results2.at[i, "rank_runoffs"] = rc.runoffs

    results2.at[i, "rank_margins"] = rc.rank_margin
    results2.at[i, "rank_margins_pers"] = round(rc.rank_margin/rc.total_votes, 3)

    results2.at[i, "rank_fav_wins"] = rc.rank_fav_win

rfws = results1["rank_fav_wins"].sum()
print(f"0.5 Turnout Percentage Rank Choice Election Fav. Wins Sum: {rfws}/{n}")

rm = round(results1["rank_margins"].mean(), 3)
print(f"0.5 Turnout Percentage Rank Choice Election Win Margin: {rm}")
rpm = round(results1["rank_margins_pers"].mean(), 3)
print(f"0.5 Turnout Percentage Election Win Margin Percentage: {rpm}%")


rfws = results2["rank_fav_wins"].sum()
print(f"0.25 Turnout Percentage Rank Choice Election Fav. Wins Sum: {rfws}/{n}")

rm = round(results2["rank_margins"].mean(), 3)
print(f"0.25 Turnout Percentage Rank Choice Election Win Margin: {rm}")
rpm = round(results2["rank_margins_pers"].mean(), 3)
print(f"0.25 Turnout Percentage Rank Choice  Election Win Margin Percentage: {rpm}%")


sns.histplot(data=results1, x="rank_margins")
plt.show()
sns.histplot(data=results1, x="rank_margins_pers")
plt.show()
sns.histplot(data=results1, x="rank_runoffs")
plt.show()

sns.histplot(data=results2, x="rank_margins")
plt.show()
sns.histplot(data=results2, x="rank_margins_pers")
plt.show()
sns.histplot(data=results2, x="rank_runoffs")
plt.show()

  0% (1 of 1000) |                       | Elapsed Time: 0:00:02 ETA:   0:45:17

KeyboardInterrupt: 